<a href="https://colab.research.google.com/github/jfjoung/AI_For_Chemistry/blob/main/notebooks/week5/Week_5_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


🎯 학습 목표:
- 비지도 학습의 기본 개념과 군집화 알고리즘의 주요 원리를 이해한다.
- K-평균 클러스터링 및 계층적 군집화 알고리즘을 학습하고, 이를 사용하여 데이터를 군집화하는 방법을 익힌다.
- 군집화 결과의 평가 방법을 학습하고, 다양한 평가 지표를 사용하여 군집화 성능을 분석한다.
- 고급 군집화 기법인 DBSCAN을 사용하여 데이터의 구조를 탐색하는 방법을 배운다.
- 비지도 학습을 활용하여 새로운 데이터에 대한 통찰을 얻고, 실제 문제에서 군집화 기법을 효과적으로 적용할 수 있는 방법을 학습한다.


In [ ]:
! pip install scikit-learn-extra rdkit plotly

!mkdir -p data/
!wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week5/unknown_clusters.csv -O data/unknown_clusters.csv
!wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week5/utils.py -O utils.py

In [ ]:
from sklearn.datasets import make_blobs
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd

from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from sklearn.metrics import pairwise_distances

from rdkit.Chem import AllChem, Descriptors, MolFromSmiles, rdMolDescriptors
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

from utils import plot_3d, plot_2d

## Clustering

> Clustering은 **유사한 데이터 포인트**를 특징이나 속성에 따라 **그룹화**하는 강력한 비지도 학습 기법입니다.

이를 통해 **대규모 복잡한 데이터셋의 내재된 구조**에 대한 통찰을 얻고, **즉시 드러나지 않는 패턴**과 관계를 **식별**할 수 있습니다.

이 노트북에서는 네 가지 군집화 알고리즘을 살펴봅니다:

- KMeans
- KMedoids
- DBSCAN
- Butina 군집화

이 알고리즘들은 화학을 포함한 다양한 분야에서 널리 사용되며, 연구자들이 데이터를 더 잘 이해하고 정보에 기반한 결정을 내리는 데 도움을 줄 수 있습니다.

### 이 노트북에서 할 일:

- [ ] 합성 데이터셋(toy example)을 생성합니다.
- [ ] 각 알고리즘을 합성 데이터셋에 적용합니다.
- [ ] 군집화 성능을 평가하기 위해 군집화 지표를 사용합니다.
- [ ] 이를 실제 화학 데이터셋에 적용합니다.

마지막 연습에서는 실루엣 점수(silhouette)와 관성(inertia)과 같은 지표를 사용하여 이상적인 군집 수를 찾습니다.

이 노트북을 끝내면, **군집화가 어떻게 작동하는지**와 **실제 문제에 다양한 군집화 알고리즘을 어떻게 적용할 수 있는지**에 대해 더 잘 이해하게 될 것입니다.

---


### synthetic clustering 데이터셋을 생성해보겠습니다.

우리는 scikit-learn의 *make_blobs* 함수를 사용할 것입니다.

*make_blobs* 함수는 가우시안 분포를 가진 랜덤 데이터 포인트를 생성합니다. 이 데이터 포인트들은 군집으로 생성되며, 각 군집은 유사한 특징을 가진 데이터 포인트들의 그룹을 나타냅니다.

이 함수는 `X`와 `y` 배열을 반환합니다. `X`는 각 데이터 포인트의 좌표를 포함하고, `y`는 각 데이터 포인트가 속한 군집을 나타내는 레이블을 포함합니다.

*make_blobs* 함수의 매개변수 값을 다르게 설정함으로써 군집의 특성, 군집의 개수, 특징, 표준 편차 등이 다른 합성 데이터셋을 생성할 수 있습니다.

이 데이터셋들은 다양한 군집화 알고리즘을 평가하는 데 사용될 수 있습니다.


In [ ]:
# 합성 군집화 데이터를 생성합니다.
n_clusters = 4  # 군집의 개수
n_features = 512  # 각 데이터 포인트의 특징 수

# 사용자 정의 군집 표준 편차 설정
cluster_stds = [0.5, 1.5, 1, 2.0]  # 각 군집의 표준 편차
n_samples = [200, 300, 100, 150]  # 각 군집의 샘플 수
cluster_centers = np.random.randint(-5, 5, size=(n_clusters, n_features))  # 군집 중심 설정

# make_blobs 함수를 사용하여 합성 데이터셋 생성
X, y = make_blobs(n_samples=n_samples, centers=None, cluster_std=cluster_stds, n_features=n_features, center_box=(-1, 1))

# 데이터를 이진 형식으로 변환하여 분자 핑거프린트를 에뮬레이트합니다
X_binary = np.where(X > 0, 1, 0)  # X 값이 0보다 크면 1, 아니면 0으로 변환


In [ ]:
# PCA를 사용하여 플로팅을 위한 좌표를 얻습니다.
pca = PCA(n_components=3)  # 3개의 주성분으로 차원 축소
coords = pca.fit_transform(X_binary)  # X_binary 데이터를 PCA를 사용해 3차원으로 변환


## Clustering algorithms

### K-means

> **k-means**는 머신러닝에서 가장 널리 사용되는 군집화 알고리즘 중 하나입니다.

이를 사용하려면 **군집의 수** `k`를 **선택**해야 합니다.

- 알고리즘은 먼저 특징 공간에서 `k`개의 중심점(centroid)을 무작위로 선택하고, 데이터셋의 각 포인트를 *가장 가까운 중심점*에 할당하여 `k`개의 군집을 정의합니다.

- 그 다음, 각 군집의 평균을 사용하여 새로운 중심점을 계산하고, 각 포인트를 가장 가까운 중심점에 다시 할당합니다.

- 이 과정은 중심점이 더 이상 변화하지 않거나 미리 정해진 반복 횟수에 도달할 때까지 계속됩니다.

> KMeans는 이미지 분할, 시장 세분화, 데이터 마이닝 등 다양한 분야에서 널리 사용됩니다. 이는 간단하면서도 강력한 알고리즘으로, 대규모 데이터셋을 효율적으로 군집화할 수 있습니다. 그러나 중심점 초기화에 따라 성능이 크게 달라질 수 있으며, 비구형(non-spherical)이나 겹치는 군집을 가진 데이터셋에는 잘 작동하지 않을 수 있습니다.


In [ ]:
# n_clusters를 설정하여 KMeans 방법을 정의합니다.
kmeans = KMeans(n_clusters=n_clusters, n_init=10, init='k-means++')  # n_init=10은 초기화 반복 횟수, init='k-means++'는 더 나은 초기 중심점 선택 방법
kmeans.fit(X_binary)  # X_binary 데이터에 대해 KMeans 알고리즘을 학습시킵니다


우리는 **k-means의 결과**를 데이터셋에 대해 시각화할 수 있습니다. 데이터는 저차원 공간으로 투영하고, 군집에 따라 데이터 포인트를 색칠합니다.

> 이 작업을 위한 코드는 `utils.py`에 작성되어 있습니다.


In [ ]:
# 두 개의 서브플롯을 생성하여 KMeans 결과를 시각화합니다.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯 생성

# 실제 군집을 2D로 플로팅
plot_2d(coords, y, title="Actual Clusters", ax=ax1)

# KMeans로 예측된 군집을 2D로 플로팅
plot_2d(coords, kmeans.labels_, title="KMeans Predicted Clusters", ax=ax2)

plt.show()  # 그래프를 화면에 표시


우리는 **k-means가 네 개의 뚜렷한 군집을 찾을 수 있음을** 확인할 수 있습니다. 이 군집들은 make_blobs 함수로 생성된 실제 군집과 밀접하게 일치합니다.

**특정 색상(레이블)은 중요하지 않다는 점을 유의하십시오. 중요한 것은 알고리즘이 서로 다른 군집을 서로 구분할 수 있다는 점입니다.**

--

때때로 2D로 데이터를 보는 것만으로는 내재된 구조와 패턴을 완전히 이해하기 어려울 수 있습니다.

그럴 경우, 더 많은 PCA 컴포넌트를 사용하여 **데이터를 3D로 플로팅**할 수 있습니다.


In [ ]:
plot_3d(coords, labels=kmeans.labels_, title="KMeans clustering")

### K-medoids

K-medoids는 **KMeans와 유사한 군집화 알고리즘으로, 몇 가지 중요한 차이점**이 있습니다.

k-means처럼 k-medoids는 데이터셋을 미리 정의된 `k` 개수의 군집으로 나누는 것을 목표로 합니다. 그러나 K-medoids는 각 군집의 대표 점으로 *중심점(centroid)* 대신 *메도이드(medoids)* (즉, 각 군집에서 가장 중심에 위치한 데이터 포인트)를 사용합니다.

- 이로 인해 K-medoids는 **노이즈와 이상치에 더 강인**하며, 비구형 및 비볼록 군집도 처리할 수 있습니다.
- KMedoids 군집의 중심점은 **데이터셋에서 실제 데이터 포인트**여야 하며, k-means는 공간 내의 어떤 점을 사용해도 됩니다.

이 차이점은 아래에서 설명됩니다:

<div align="left">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/K-MeansAndK-Medoids.png" width="600"/>
</div>

K-means는 일반적으로 유클리드 거리(Euclidean distance) 지표를 사용하지만, K-medoids는 **어떤 거리 지표**든 사용할 수 있어, 다양한 유형의 데이터셋에 대해 더 유연하고 적응력이 뛰어납니다.

#### 여기에서 다양한 거리 지표를 시도해보고, 군집화가 어떻게 변화하는지 확인해보세요!

사용 가능한 옵션은 다음과 같습니다:

- euclidean
- jaccard
- cityblock
- cosine
- l2
- minkowski

> 거리 지표에 대해 더 자세히 알고 싶다면, [여기](https://medium.com/geekculture/7-important-distance-metrics-every-data-scientist-should-know-11e1b0b2ebe3)에서 읽어보세요. 그리고 모든 사용 가능한 옵션은 [여기](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html)에서 확인할 수 있습니다.


In [ ]:
# 거리 지표를 설정합니다. (변경 가능)
d_metric = 'euclidean'  # 거리 지표를 원하는 값으로 변경하세요.

# KMedoids 알고리즘을 정의합니다.
kmedoids = KMedoids(
    n_clusters=n_clusters,  # 군집의 개수
    random_state=42,  # 랜덤 시드 설정
    init='k-medoids++',  # 초기 중심점 설정 방법
    metric=d_metric,  # 사용할 거리 지표
    max_iter=50000  # 최대 반복 횟수
)

# X_binary 데이터에 대해 KMedoids 알고리즘을 학습시킵니다.
kmedoids.fit(X_binary)


In [ ]:
# 그리고 여기 K-medoids 결과를 시각화한 그래프입니다.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯 생성

# 실제 군집을 2D로 플로팅
plot_2d(coords, y, title="Actual Clusters", ax=ax1)

# K-medoids로 예측된 군집을 2D로 플로팅
plot_2d(coords, kmedoids.labels_, title=f"K-medoids Predicted Clusters\n(Distance = {d_metric})", ax=ax2)

plt.show()  # 그래프를 화면에 표시


In [ ]:
plot_3d(coords, labels=kmedoids.labels_, title=f"KMedoids clustering\n(Distance = {d_metric})")

### DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

DBSCAN은 밀도 기반 군집화 알고리즘으로, 데이터 포인트들을 밀도에 따라 그룹화합니다.

이 알고리즘은 **임의 모양의 군집**을 찾을 수 있으며, **노이즈에 강인**합니다. K-means나 K-medoids와 달리, DBSCAN은 **군집의 개수를 미리 지정할 필요가 없습니다**.

DBSCAN의 핵심 아이디어는 군집이 다른 밀집된 지역들과 낮은 밀도의 영역들에 의해 구분된 **밀집된 포인트들의 영역**이라는 것입니다.

DBSCAN은 두 가지 매개변수를 요구합니다:

- eps (epsilon): 두 포인트가 이웃으로 간주되기 위한 최대 거리.
- min_samples: 밀집된 영역을 형성하기 위한 최소 포인트 수(핵심 포인트).

이 알고리즘은 각 데이터 포인트 주변에 이웃을 정의하고, eps 매개변수에 따라 서로 가까운 포인트들을 그룹화합니다. 이웃에 최소 min_samples 포인트가 포함되어 있으면, 그 포인트는 핵심 포인트로 간주됩니다. 핵심 포인트에서 도달할 수 있는 포인트들은 같은 군집에 속하게 됩니다. 핵심 포인트에 도달할 수 없는 포인트들은 노이즈로 처리됩니다.


In [ ]:
# DBSCAN 알고리즘을 정의하고 학습시킵니다.
dbscan = DBSCAN(eps=0.5, min_samples=5, metric='jaccard')  # eps=0.5, min_samples=5, 거리 지표는 'jaccard'로 설정
dbscan.fit(X_binary)  # X_binary 데이터에 대해 DBSCAN 알고리즘을 학습시킵니다


In [ ]:
# DBSCAN 결과를 시각화합니다.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯 생성

# 실제 군집을 2D로 플로팅
plot_2d(coords, y, title="Actual Clusters", ax=ax1)

# DBSCAN으로 예측된 군집을 2D로 플로팅
plot_2d(coords, dbscan.labels_, title="DBSCAN Predicted Clusters", ax=ax2)

plt.show()  # 그래프를 화면에 표시


### Butina Clustering Algorithm

Butina 군집화 알고리즘은 **이진 특징을 가진 대규모 데이터셋**을 처리하기 위해 설계되었습니다. 이는 특히 **화학 정보학에서 분자 핑거프린트**(분자 구조의 이진 표현)를 군집화하는 데 유용합니다.

Butina 군집화 알고리즘은 **단일 연결 계층 군집화 방법**으로, 사용자 정의 유사성 임계값(cutoff)을 기준으로 군집을 병합합니다. 더 자세한 내용은 [논문에서](https://pubs.acs.org/doi/full/10.1021/ci9803381) 확인할 수 있습니다.

> 다른 군집화 방법에 비해 주요 장점 중 하나는 이진 데이터와 비유클리드 거리 지표(Jaccard와 같은)를 잘 처리할 수 있다는 점입니다.

Butina 군집화의 cutoff 매개변수를 설정하려면 다음 단계를 따르세요:

1. 데이터셋에 대해 쌍별 거리(Jaccard 또는 다른 적합한 거리 지표)를 계산합니다.

2. 거리를 시각화하여 히스토그램을 작성합니다.

3. 히스토그램을 분석하고 유사한 데이터 포인트와 비유사한 데이터 포인트를 구분할 수 있는 합리적인 임계값에 해당하는 cutoff 값을 선택합니다.

적절한 cutoff 값을 선택한 후, 이를 Butina 군집화 알고리즘의 입력 매개변수로 사용하면 됩니다.

#### 다양한 cutoff 값을 실험하여 데이터에 가장 만족스러운 군집화 결과를 도출해보세요.


In [ ]:
# Jaccard 거리로 쌍별 거리 계산
distances = pairwise_distances(X_binary.astype(bool), metric='jaccard')  # X_binary를 불리언 타입으로 변환 후 Jaccard 거리 계산

# 거리 분포를 히스토그램으로 시각화
plt.hist(distances.flatten(), bins=50)  # 히스토그램을 50개의 구간으로 생성
plt.xlabel("Jaccard Distance")  # x축 레이블
plt.ylabel("Frequency")  # y축 레이블
plt.show()  # 그래프를 화면에 표시


In [ ]:
class ButinaClustering:
    def __init__(self, cutoff=0.8, metric='jaccard'):
        self.cutoff = cutoff  # 임계값(cutoff) 설정
        self.metric = metric  # 사용할 거리 지표 설정

    def fit(self, x):
        """
        분자 핑거프린트 집합에 대해 Butina 군집화 수행

        :param x: 이진 데이터의 numpy 배열
        :return: self
        """
        # 거리 행렬 계산
        distance_matrix = []
        x = x.astype(bool)  # x 데이터를 불리언 형식으로 변환
        for i in range(1, len(x)):
            distances = pairwise_distances(x[i,:].reshape(1, -1), x[:i,:], metric=self.metric)  # 각 데이터 포인트 간 거리 계산
            distance_matrix.extend(distances.flatten().tolist())  # 계산된 거리를 리스트에 추가

        # Butina 군집화 수행
        clusters = Butina.ClusterData(distance_matrix, len(x), self.cutoff, isDistData=True)  # 군집화
        self.clusters = clusters  # 군집 결과 저장

        # 각 데이터 포인트에 군집 레이블 할당
        cluster_labels = np.full(len(x), -1, dtype=int)  # 초기 군집 레이블은 -1로 설정
        for label, cluster in enumerate(clusters):  # 군집을 순차적으로 탐색
            for index in cluster:
                cluster_labels[index] = label  # 해당 군집에 속하는 데이터 포인트에 레이블 할당

        self.labels_ = cluster_labels  # 최종 군집 레이블 저장


    def fit_predict(self, x):
        self.fit(x)  # 군집화 수행
        return self.labels_  # 군집화 레이블 반환


In [ ]:
cutoff = 0.65  # 임계값(cutoff)을 설정합니다. (이 값을 변경할 수 있습니다)

butina = ButinaClustering(cutoff=cutoff, metric='jaccard')  # ButinaClustering 객체 생성
butina.fit(X_binary)  # X_binary 데이터에 대해 군집화 수행

print(f"{len(butina.clusters)} clusters were found with a cutoff = {cutoff}")  # 군집 개수 출력


In [ ]:
# Butina 군집화 결과를 시각화합니다.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯 생성

# 실제 군집을 2D로 플로팅
plot_2d(coords, y, title="Actual Clusters", ax=ax1)

# Butina 군집으로 예측된 군집을 2D로 플로팅
plot_2d(coords, butina.labels_, title=f"Butina Predicted Clusters\n(cutoff = {cutoff})", ax=ax2)

plt.show()  # 그래프를 화면에 표시


## Evaluation

위의 연습들에서 알 수 있듯이, 모든 알고리즘은 **다른 결과**를 제공하며, 그 결과는 또한 **선택한 매개변수에 따라 달라집니다**. 그럼 질문은,

> 알고리즘의 결과가 좋은지 어떻게 알 수 있을까요?

다음 섹션에서는 **inertia**와 **silhouette** 점수와 같은 군집화 지표들이 K-means 또는 K-medoids 알고리즘을 사용할 때 최적의 군집 수를 찾아내는 데 어떻게 도움이 되는지 평가할 것입니다.

### Inertia
Inertia는 각 데이터 포인트와 그 데이터 포인트가 할당된 군집의 중심점 간의 제곱 거리 합입니다.

> Inertia는 군집 내 데이터 포인트들이 얼마나 밀집되어 있는지를 측정하는 지표입니다.

**낮은 inertia 값**은 군집 내 데이터 포인트들이 중심점에 더 가깝다는 것을 의미하며, 이는 **바람직한 결과**입니다.

하지만 inertia는 **군집의 수에 민감**할 수 있으므로, 군집 수가 증가하면 일반적으로 inertia 값이 줄어듭니다.
**따라서 inertia만으로 최적의 군집 수를 선택하면 과적합이 발생할 수 있습니다**.

### Silhouette Score

> Silhouette score는 데이터 포인트가 다른 군집과 비교해 자신이 속한 군집과 얼마나 유사한지를 측정하는 지표입니다.

Silhouette score는 -1에서 1까지의 값을 가집니다. **높은 silhouette score**는 **데이터 포인트가 자신의 군집에 잘 맞고** 이웃 군집과는 잘 맞지 않음을 나타냅니다. 음수 silhouette score는 데이터 포인트가 잘못된 군집에 배정되었음을 시사합니다.

Silhouette score는 **inertia보다 더 강력**하게 최적의 군집 수를 결정할 수 있습니다. 이는 군집 내 응집도(데이터 포인트들이 군집 내에서 얼마나 밀접한지)와 분리도(군집들이 서로 얼마나 구별되는지)를 모두 고려하기 때문입니다.

---

inertia와 silhouette 점수를 비교하고 데이터를 위한 최적의 군집 수를 결정하려면 다음 단계를 따르세요:

1. 군집 수의 범위(예: 2에서 10까지)를 반복하면서 각 군집 수에 대해 군집화 알고리즘(KMeans, KMedoids 등)을 적합시킵니다.
2. 각 군집화 모델과 각 군집 수에 대해 inertia와 silhouette 점수를 계산하고 저장합니다.
3. 군집 수에 따른 inertia와 silhouette 점수를 플로팅합니다.
4. 플롯을 분석하여 최적의 군집 수를 결정합니다.

> Inertia 플롯에서는 inertia 값이 완만하게 감소하기 시작하는 "엘보" 지점을 찾아보세요.
> Silhouette 플롯에서는 가장 높은 silhouette 점수를 찾아보세요.


In [ ]:
n_clusters = 4  # 군집의 개수 설정
algorithms = {
    'KMeans': KMeans(n_clusters=n_clusters, n_init=10, init='k-means++', random_state=42),  # KMeans 알고리즘 설정
    'KMedoids': KMedoids(n_clusters=n_clusters, init='k-medoids++', metric='jaccard', random_state=42)  # KMedoids 알고리즘 설정
}


In [ ]:
def plot_inertia_and_silhouette(data, algorithms, min_clusters, max_clusters):
    # 주어진 알고리즘과 군집 수 범위에 대해 inertia와 silhouette 점수를 플로팅하는 함수

    for name, algorithm in algorithms.items():  # 알고리즘마다 반복
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))  # 1행 2열의 서브플롯 생성
        fig.suptitle(f'{name}')  # 알고리즘 이름을 제목으로 설정

        inertia = []  # inertia 값을 저장할 리스트
        silhouette_scores = []  # silhouette 점수를 저장할 리스트
        for n_clusters in range(min_clusters, max_clusters + 1):  # 군집 수 범위에 대해 반복
            algorithm.set_params(n_clusters=n_clusters)  # 군집 수 설정
            labels = algorithm.fit_predict(data)  # 군집화 실행
            inertia.append(algorithm.inertia_)  # inertia 값 추가
            silhouette_scores.append(silhouette_score(data, labels))  # silhouette 점수 추가

        # inertia 값 플로팅
        ax1.plot(range(min_clusters, max_clusters + 1), inertia, label=name)
        # silhouette 점수 플로팅
        ax2.plot(range(min_clusters, max_clusters + 1), silhouette_scores, label=name)

    # 플로팅 설정
    ax1.set_xlabel("Number of clusters")  # x축 레이블
    ax1.set_ylabel("Inertia")  # y축 레이블
    ax1.legend()  # 범례 추가

    ax2.set_xlabel("Number of clusters")  # x축 레이블
    ax2.set_ylabel("Silhouette Score")  # y축 레이블
    ax2.legend()  # 범례 추가

    plt.show()  # 그래프를 화면에 표시


In [ ]:
min_clusters = 2
max_clusters = 10

plot_inertia_and_silhouette(X_binary.astype(bool), algorithms, min_clusters, max_clusters)

---

# Exercise

이제 K-means 또는 K-medoids 알고리즘과 inertia vs. silhouette 지표를 사용하여 화학 데이터셋에서 최적의 군집 수를 찾아볼 차례입니다.

`plot_inertia_and_silhouette` 함수를 사용하여 `unknown_clusters` 데이터셋에서 적절한 군집 수를 추정하세요.

- `data/unknown_clusters.csv`에서 데이터셋을 읽어옵니다. (변수명: `data_ex` 사용)
- SMILES를 Morgan fingerprints로 특징화합니다. (변수명: `X_ex`에 특징화된 SMILES 저장)
- `plot_inertia_and_silhouette` 함수를 실행하여 최적의 군집 수를 추정합니다.


In [ ]:
# 1. 데이터셋 읽기
data_ex = pd.read_csv('data/unknown_clusters.csv')  # 데이터셋 로드

# 2. SMILES를 Morgan fingerprints로 featurize
def featurize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)  # Morgan fingerprint 생성
    return list(fingerprint)

X_ex =  # SMILES 컬럼을 Morgan fingerprints로 변환

# 3. 데이터를 Numpy 배열로 변환 및 표준화


# 4. K-means 또는 K-medoids 알고리즘으로 최적 군집 수 추정


# 5. inertia와 silhouette 점수 시각화 함수 실행

#Solution
# %load https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/notebooks/week5/solution_05.py


In [ ]:
# YOUR CODE
N_CLUSTERS =

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from rdkit import Chem
from rdkit.Chem import Draw

# Perform clustering using KMeans and KMedoids
kmeans = KMeans(n_clusters=N_CLUSTERS, n_init=10, random_state=42).fit(X_ex)
kmedoids = KMedoids(n_clusters=N_CLUSTERS, init='k-medoids++', metric='jaccard', random_state=42).fit(X_ex.astype(bool))


# Function to select a few representative molecules from each cluster
def plot_representative_molecules(labels, smiles, n_clusters, n_molecules=5):
    for i in range(n_clusters):
        cluster_indices = np.where(labels == i)[0]
        molecules = [Chem.MolFromSmiles(smile) for smile in smiles]
        cluster_molecules = [molecules[idx] for idx in cluster_indices]

        # Select the first n_molecules from the cluster
        selected_molecules = cluster_molecules[:n_molecules]

        # Plot the selected molecules
        img = Draw.MolsToGridImage(selected_molecules, molsPerRow=n_molecules, subImgSize=(200, 200))
        print(f"Cluster {i+1}:")
        display(img)

# Plot the representative molecules for KMeans
print("KMeans Clusters:")
plot_representative_molecules(kmeans.labels_, data_ex['smiles'], N_CLUSTERS)

# Plot the representative molecules for KMedoids
print("KMedoids Clusters:")
plot_representative_molecules(kmedoids.labels_, data_ex['smiles'], N_CLUSTERS)
